![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_red.png?raw=true)
       
# Topic 1: Introduction to Asynchronous Programming in Javascript       
       
![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_red.png?raw=true)

<br />

* Several years ago before AJAX in Javascript is popular, websites that want to fetch data from servers, have to done this through PHP


* The downside to PHP is that it is a server-side programming language. Everytime a data is required, a request is sent to server, which sends back the new processed HTML document. This requires the webpage to be refreshed everytime a request is made, which can be frustrating to the users.


* Now, __AJAX (Asynchronous Javascript and XML)__ is very available to be used in webpages that possess the ability to retrieve data from the server without having to refresh the entire page!


<br />

---

<br />

* What we know so far is that Javascript is a single-threaded programming language. It execute the codes line by line. 


* However, sending the request, processing the request by the server and the data sending back all takes certain unpredictable amounts of time. Problem is, how can Javascript, being a synchronous language, will be able to essentially 'pause' and wait for the data to come back, and then only to run some code?

<br />

---

<br />

* The answer actually lies within the browser itself. Javascript is synchronous, but the browser can do more than just that.


* Javascript executes codes on top of the __Callstack__, which is essentially a Stack of instructions awaiting to be executed by the browser. The instructions on the top of callstack will keep being executed and popped.


* Now, when the callstack finds out the code requires some time to pass before executing (Can be simple `setTimeout` or `fetch` requests), it will notify the browser to handle that code.


* The asynchronous code will be pushed to the __browser's web API__ and removed from the callstack to resume normal execution of the rest of codes


* Once the time is up, the broswer will push the code to be executed, along with the new data (if any) back to the callstack, so that the code is executed next


* This way, the effect of multithreading is achieved although Javascript being a synchronous language!

<br />

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_red.png?raw=true)

## 1.1 - Looking the Effects of Asynchronous Behavior

<br />

* The following code will simulate a data fetching from a server, which takes certain amount of time. Let's see what happens if we didn't know about concept of asynchronous Javascript and AJAX!

In [ ]:
%%js

function fetchUser() {
    let result;
    setTimeout(()=>{
        element.append("Data retrieved successfully!" + "<br>" );
        result = {
            name: "John Wick",
            gender: "M"
        };
    },3000);
    return result;
}


element.append("The user is: " + fetchUser() + "<br>" );

<br />

* We can observe that the very first message printed is `The user is: undefined`, which is supposed to be printed after message `Data retrieved successfully!`. This is not what we want!


* The reason this occurred is because when we call on the function `fetchUser()`, the `setTimeout` is executed, the code is pushed to the browser's Web API and waiting for 3 seconds to pass, and immediately after that (before 3 seconds had passed), the `return` is already executed, which returns `undefined`

<br />


<br />

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

# 2.0 - Asynchronous Programming in Javascript

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

<br />

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

##  2.1 - Callback function

<br />

* Before 2015 when the ES6 is released, developers would use what's known as __callback functions__ to perform some code after the data is retrieved.


* A __callback function__ is essentially __a function that is passed into another function as a argument, and executed inside that function__.


* The `setTimeout()` function is already an example of a function that takes in a callback function as its argument. After the specified milliseconds had passed, only then the callback function would be executed (which in most cases we use the anonymous arrow function)


* Therefore we can create a callback function that __handles the data once it is received__, and __only call the function when we 100% sure the data is retrieved__.

<br />

In [ ]:
%%js

// Now the function takes in a callback function as argument, and thus can execute
// it once data is ensured to be retrieved
function fetchUser( callback ) {
    setTimeout(()=> {
        element.append("User successfully retrieved!" + "<br>");
        let user = {
            name: "John Wick",
            gender: "M"
        };
        callback( user );      // Execute the callback with data passed in
    }, 3000);
}


fetchUser( (user)=> element.append(user.name + "<br>")  );

// We want to print the User's gender
fetchUser( (user)=> element.append(user.gender + "<br>")  );

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

## 2.2 Callback Hell

<br />

* Now the callback function method looks okay to be used. However, what if the operations is meant to be chained?


* For example, after fetching the user, I want to fetch the user's friends. After fetching the user's friends, I want to fetch the user's friend profile picture?


* This means the callback function essentially has to take in another callback function, which in turn, also need to take in yet another callback function! 


* This causes kind of a messy, hard to read and hard to maintain code. Developers known it as __callback hell__

<br />

In [ ]:
%%js


function fetchUser(callback) {
    element.append("Now fetching user..." + "<br>" );
    setTimeout(()=> {
        let user = { name: "John Wick", gender: "M" };
        element.append("User retrieved: " + user.name + "<br>" );
        callback(user);
    },3000);
}

function fetchUserFriend(user, callback) {
    element.append("Now fetching friends of " + user.name + "..." + "<br>" );
    setTimeout(()=> {
        let friends = ["Bob", "Kahn", "Peter"];
        element.append("Friends retrieved: " + friends + "<br>" );
        callback(friends);
    }, 3000);
}

function fetchProfilePicture(user) {
    element.append("Fetching profile pic of " + user + "..." + "<br>" );
    setTimeout(()=> {
        element.append("Profile picture retrieved for " + user + "<br>" );
    }, 3000);
}


// Callback Hell Occurs here. Observe the triangular pattern, very hard to read

fetchUser( (user)=> {
    fetchUserFriend(user, (friends)=> {
        friends.forEach( (f)=> {
            fetchProfilePicture(f);
        });
    });
});

<br />

* To add salt to injury, know that server is not always up! If the fetching of data failed, we might want to have a function executed to handle the error.


* This way, the code becomes more complicated than ever. In addition of having to pass in a callback function for success retrieval of data, we also need to pass in another callback function in case of error when retrieving the data!

<br />

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

## 2.3 - ES6 - Promises

<br />

* Developers of Javascript understand the issue faced by programmers, and thus introduced __`Promises`__ in Javascript's ES6 update.


* Essentially, the callback hell is now replaced by a more easier read chained statements, just like in functional programming - pipelined operations.


* Asynchronous programming is then done using `Promises`, which is a class for handling asynchronous processes.

<br />

* The `Promise` is instantiated using the following syntax
```javascript
    let myPromise = new Promise( (resolve, reject) => {
        // code to fetch data asynchronously
        if data is fetched {
            resolve( data );
        }
        else if something goes wrong {
            reject( error );
        }
    });
```
<br />

* The `Promise` takes in a callback function as argument, which itself takes in 2 callback function `resolve` and `reject` (To avoid confusion, `resolve` and `reject` is not defined by programmer. It is automatically assigned upon instantiation of object)


* Inside the callback function, we write the codes to fetch our data asynchronously. Upon fetched, we can then use the `resolve` function to indicate the data is successfully retrieved, or `reject` function to indicate unable to fetch data and return an error.

<br />

---

<br />

* When we have just instantiated the `Promise` object, the code still won't run. We essentially have to use the object's method `.then( callback )`. It takes in a callback function which requires an argument of the data retrieved.


* What `.then()` does is to execute the callback function consisting of the asynchronous code we defined in the constructor, and through the `resolve` method, it will obtain the data fetched.


* In the `.then()` method, we pass in a callback function as argument, which operates on the data fetched. Then, the return value of this passed callback function will be used by the `.then()` method to create an entirely new `Promise` object, and returned.


* Since an entirely new `Promise` object is returned, this means we can chain operations of `Promises`! No more callback hell!

<br />

---

<br />

* How about error when fetching the data? Apart from the `.then()` method which handles successfully retrieved data through `resolve()` method, we also have one which handles error from `reject()` : `catch( callback )`


* When a Promise had the `reject()` code ran, it will essentially pass through the `.then()` statements without executing everything, until it had reached the `.catch()` statement, which error is handled


* Therefore it is a good practice to leave the `.catch()` until the end of the chain so that all errors are handled in one place after the `.then()` statements.

<br />

---

<br />

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

## 2.4 - Promise Returning Function

<br />

* The general design pattern is to have a __Promise returning function__. Essentially when the function is executed, it immediately returns a `Promise` object which we can chain `.then()` operations on.

<br />

In [ ]:
%%js

// The Promise returning function
function promiseReturningCounter() {
    // Mimic behavior of static variables. Since function is also an Object
    promiseReturningCounter.value = promiseReturningCounter.value + 1 || 1 ;
    let { value } = promiseReturningCounter;
    
    return new Promise( (resolve, reject) => {
       setTimeout(()=> {
           if (value > 3) reject( `Out of range: ${value}`);
           else resolve(value);
       }, 1000);
    });
}


// Use the Promise returning function
promiseReturningCounter()
.then( (v) => {
    element.append(v + "<br>" );
    return promiseReturningCounter();
}).then( (v) => {
    element.append(v + "<br>" );
    return promiseReturningCounter();
}).then( (v) => {
    element.append(v + "<br>" );
    return promiseReturningCounter();
}).then( (v) => {     // This one won't even run because by this line, the promise would end up rejected
    element.append(v + "<br>" );
    return promiseReturningCounter();
}).catch( (e) => {
    element.append(e + "<br>" );
});


<br />

* Now let's reapply this `Promise` concept into the user fetching system!

<br />

In [ ]:
%%js

// Fetch the user
function fetchUser() {
    return new Promise((resolve, reject)=>{
        setTimeout(()=> {
            let user = {name:"John Wick", gender: "M"};
            element.append(`User ${user.name} fetched!` + "<br>" );
            resolve( user );
        }, 2000);
    });
}

// Fetch the user friends
function fetchFriends( user ) {
    return new Promise((resolve, reject)=> {
        setTimeout(()=> {
            element.append(`Friends of ${user.name} fetched!` + "<br>" );
            let friends = ['Bob', "Charlie", 'Daniel'];
            resolve(friends);
        }, 2000);
    });
}

// Fetch friend's profile picture
function fetchProfilePic( user ) {
    return new Promise((resolve, reject)=> {
        setTimeout(()=> {
           element.append(`Profile picture of ${user} fetched!` + "<br>" );
        }, 2000);
    });
}

fetchUser()
.then( (user) => fetchFriends(user) )
.then( (friends) => {
    friends.forEach( (friend)=> {
        fetchProfilePic( friend ).then();
    });
});

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

## 2.5 - Promises .all(), .race(), .any()

<br />

* If we have multiple `Promise` to be executed, the most common method is to use the above methods and pass in an iterable of `Promise`. How they handle each Promises is quite different
    * `.all()` wait until all Promises are resolved, then only the returned Promise is resolved. If at least one among the Promises failed, then the returned Promise will be rejected as well.
    * `.race()` will return a `Promise` that resolves or rejects as soon as one among the Promises is resolved or rejected.
    * `any()` will return a resolved `Promise` whenever one among the Promises is resolved. Note that one Promise rejecting will not immediately cause returned Promise to reject, it will occur if __all__ of the Promises are rejected
    
<br />

In [ ]:
%%js


function fetchUser( username ) {
    return new Promise((resolve,reject)=> {
       setTimeout(()=> {
            element.append(`${username} data is fetched!` + "<br>" );
            resolve();
        }, Math.random() * 5000); 
    });
}



Promise.all( [ fetchUser("Adrian") , fetchUser("Bob"), fetchUser("Charles") ] )
.then(()=> element.append("All users fetched" + "<br>" ) )
.catch(()=> element.append("Failed to fetch all users"  + "<br>" ) );


In [ ]:
%%js


// The fetchUser has a 50% chance of failing
function fetchUser( username ) {
    return new Promise((resolve,reject)=> {
       setTimeout(()=> {
           if (Math.random() > 0.5) reject("Problem Occurred in fetching " + username);
           
            resolve(`${username} is fetched successfully`);
        }, Math.random() * 3000); 
    });
}


// In .race(), as soon as one failed, whole promise will immediately reject
Promise.race( [ fetchUser("Adrain"), fetchUser("Bob"), fetchUser("Charles") ] )
.then( (arr) => element.append(arr + "<br>" )  )
.catch( (e) => element.append(e + "<br>" )  );

In [ ]:
%%js


// The fetchUser has a 80% chance of failing
function fetchUser( username ) {
    return new Promise((resolve, reject)=> {
        setTimeout(()=> {
            if (Math.random() > 0.2) reject("Problem Occurred in fetching " + username);
            
            resolve(`${username} is fetched successfully`);
        }, Math.random() * 3000);
    });
}



Promise.any( [fetchUser("Adrian"), fetchUser("Bob"), fetchUser("Charles") ] )
.then( (v) => element.append(v + "<br>" ) )
.catch( (err) => element.append(err + "<br>" ) );

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_blue.png?raw=true)

# 2.6 - Asynchronous Javascript in Disguise - Async and Await keyword

<br />


* To allow programmers to write asynchronous codes exactly like the day-to-day synchronous codes, the developers later introduced `async` and `await` keyword, which is considered a __syntatic sugar__ above `Promises`


* `async` keyword is used on a function to declare that the function will run asynchronous code. A `async` function will have the `await` keyword permitted within
```javascript
    async function fetchUser() {...}
```


* `await` keyword is used for statements with `Promise` instances which require certain amount of time to be executed and returned. By declaring the statement to be `await`, the execution of the function will pause until the `Promise` returns a value. The value is therefore assigned back to the variable we specified
```javascript
const user = await userPromiseGenerator("John");
```


* As you can see, `await` is like the `.then()` method, which tells a `Promise` to execute. Once the `Promise` is resolved, the value is assigned to the variable `user`.


* How about errors? When a `Promise` caught an error and `.reject()`, an exception basically will be thrown from the `await` statement, therefore we caught it like normal `try...catch`!
```javascript
try {
    const val = await userPromiseGenerator("John");
} catch (e) {
    console.log(e);
}
```

<br />

---

<br />


### Gotcha's in Async and Await


[Article Link](https://zellwk.com/blog/async-await-in-loops/)

<br />


* One thing to mention, however, is the behavior of `await` keyword in loops
    * For normal loops like `while`, `for` and `for...of` loops, the await behaves as we expected. Before continuing to next iteration, the program will wait for the `Promise` to be resolve beforehand
    * For loops which take in callback functions, like `forEach`, `filter`, `map`, since the looping function is just appling the callback function to each element, this means that each element will have the callback function executed almost at the same time, resulting in quite an unexpected behavior
    * For the `foreach`, since the `foreach`'s job is just to apply callback function to each element, that job is quickly done, and execution of code will be resumed ASAP, without pausing for the Promises to be resolved.
    * Since asynchronous functions always return `Promise`, using the `map` function will map the elements to an array of `Promise`, which is yet to be resolve. Therefore to execute them, we need to use `Promise.all()` or any of equivalent. 
    
<br />

In [ ]:
%%js

function counterPromise() {
    counterPromise.val = counterPromise.val + 1 || 1;
    
    return new Promise( (resolve, reject) => {
       setTimeout(()=> {
            if (counterPromise.val < 4) resolve(counterPromise.val);
            reject(`Exceeded Range: ${counterPromise.val}`);
       },1000); 
    });
}


async function count() {
    element.append("Start Counting..." + "<br>");
    try {
        while (true) {
            const val = await counterPromise();
            element.append(val + "<br>");
        }
    } catch (e) {
        element.append(e + "<br>");
    }
    element.append("Ended Counting" + "<br>");
}


count();

In [ ]:
%%js

function fetchUser( username ) {
    return new Promise( (resolve, reject)=> {
        setTimeout(()=> {
            element.append(`${username} successfully fetched` + "<br>");
            resolve( { username, gender:"M" } );
        }, 2000);
    });
}


function fetchFriends( user ) {
    return new Promise( (resolve, reject) => {
       setTimeout(()=> {
           element.append(`${user.username}'s friends successfully fetched!` + "<br>");
           resolve( ["John", "Bob", "Charles"] );
       }, 2000);
    });
}
                       
                    
function fetchProfilePic( user ) {
    return new Promise( (resolve, reject)=> {
        setTimeout(()=> {
            element.append(`Profile picture of ${user} successfully fetched!` + "<br>" );
            resolve(user);
        }, 1000);
    });
}


async function login( username ) {
    const user = await fetchUser(username);
    const friends = await fetchFriends( user );
    
    // Using for loop, each one is executed in series, one after another
    for (let f of friends)
        await fetchProfilePic(f);
    
    // See the difference of forEach, which each one is executed simultaneously?
    friends.forEach( async (f) => {
        await fetchProfilePic(f);
    });
}

login( "Adam" );

<br />

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_green.png?raw=true)

# 3.0 - AJAX - Asynchronous Javascript and XML

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_green.png?raw=true)

<br />

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_green.png?raw=true)

## 3.1 - XMLHttpRequest

[XMLHttpRequest Documentation](https://developer.mozilla.org/en-US/docs/Web/API/XMLHttpRequest)

<br />


* Back earlier before `Promise` was introduced when programmers still use callback functions for asynchronous purposes, developers use `XMLHttpRequest`, a web API to retrieve data from servers or URL without having the page to refresh like in PHP. 


* Data retrieved from servers or internet can be of various types. Common ones are JSON or pure text files. XML is popular back then, but slowly replaced by JSON.


* `XMLHttpRequest` is a class and need to be instantiated first. Only then we can setup and specify the URL and the headers that has to be sent, and what action needs to be done after data is retrieved.
    * `const xhr = new XMLHttpRequest();` - Instantiate a new `XMLHttpRequest` object
    * `xhr.open( method, url [, async, user, password] )` - Initialize a XMLHttpRequest. This is the place where we pass in the method used (`POST`, `GET` etc), and the target url
    * `xhr.onload` - We have to assign this some callback function. Upon the data is loaded, it will call the callback function with the retrieved data passed in.
    * `xhr.send()` - Sends the request
    * `xhr.setRequestHeader( header, value )` - Sets a HTTP request header with specified value
    
<br />

In [ ]:
%%js

//======================================
// Example for Text File (CSV) fetching
//======================================

const xhr = new XMLHttpRequest();
xhr.open('GET', 'https://raw.githubusercontent.com/AdmiJW/Miscellaneous-Items/master/fcc-forum-pageviews.csv');;

xhr.onload = () => {
    if (xhr.status === 200) 
        element.append( xhr.responseText.substr(0, 300).replaceAll("\r\n", "<br>") )
    else
        element.append("Error occurred while fetching data from " + xhr.responseURL + "<br>" );
}

xhr.send();

In [ ]:
%%js

//======================================
// Example for BLOB (Binary Large Objects)
//======================================

const xhr = new XMLHttpRequest();
xhr.open('GET', 'https://freesound.org/data/previews/466/466998_285997-lq.mp3');

xhr.responseType = 'blob';      // Important! So that the response can be converted into BLOB

xhr.onload = () => {
    if (xhr.status === 200) {
        element.append(xhr.getResponseHeader('Content-Type') + "<br>" );
        
        const audioHTML = document.createElement('audio');
        audioHTML.controls = true;
        element.append(audioHTML);
        
        let blob = new Blob( [ xhr.response ], {type: xhr.getResponseHeader('Content-Type')} );
        let fr = new FileReader();
        fr.onload = () => {
            audioHTML.src = fr.result;
        }
        fr.readAsDataURL(blob);
    }
    else
        element.append("Error occurred while fetching data from " + xhr.responseURL + "<br>" );
}

xhr.send();

In [ ]:
%%js

//=============================================
// Example for JSON (Javascript Object Notation)
//=============================================

const xhr = new XMLHttpRequest();
xhr.open("GET", 'https://jsonplaceholder.typicode.com/posts/1/comments');

xhr.onload = ()=> {
    if (xhr.status === 200) {
        const json = JSON.parse( xhr.response );
        
        for (let k in json ) {
            const post = json[k];
            const html = `<div><h3>${post.name}</h3><strong>${post.email}</strong><p>${post.body}</p></div><br/>`;
            element.append(html);
        }
        
    }
}

xhr.send();

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_green.png?raw=true)

## 3.2 - XMLHttpRequest with PHP


<br>

* Certainly, PHP is one of the most popular backend lannguages. Whenever we are trying to obtain some data from the server, we can done so through the PHP itself.


* Therefore, the AJAX requests can also be sent to PHP links. In that case, the output of the PHP file will be sent back and handled by Javascript, without having to actually refresh the page!


<br>

---

<br>

* However, since now the form data is not sent by browser due to `e.preventDefault()`, it means we have to encode all the data to be sent to the server manually, by ourselves


* For GET requests, in the URL to be put into the `xhr.open()` method, we need to manually include the parameters in the URL to be sent.


* For POST requests, first we have to set the request header to have the following:
```javascript
xhr.setRequestHeader('Content-type', 'application/x-www-form-urlencoded');
```
then, in the `xhr.send()` method, it accepts a optional parameter to specify the request's body. We should pass in the POST parameters like `xhr.send('submitted&name=JohnDoe&password=1234')`

<br>



In [ ]:
%%html

<!---------------------------------------------------
Example for PHP
We use APACHE local server, and assumed that
we does have the process.php lying in our server
------------------------------------------------------>

<form method='GET' id='echo_name'>
    <input type='text' name='name' placeholder='Enter name here...'/>
    <button type='submit'>Submit</button>
</form>

<div id='response_area'>
</div>



<script>
    
document.getElementById('echo_name').addEventListener('submit', (e) => {
    e.preventDefault();
    
    const name = e.srcElement[0].value;
    
    const xhr = new XMLHttpRequest();
    xhr.open('GET', 'http://127.0.0.1/Javascript%20AJAX%20Testing/testerphp.php?echo-name=&name=' + name);
    xhr.onload = ()=> {
        document.getElementById('response_area').innerHTML = xhr.response;
    }
    xhr.send();
});

</script>

In [ ]:
%%html

<!---------------------------------------------------
Example for PHP
Sending POST request
------------------------------------------------------>

<form method='POST' id='post-name-password' name='post-name-password'>
    <input type='text' name='name' placeholder='Enter username' required/>
    <input type='password' name='password' placeholder='Enter password' required/>
    <button type='submit' >Submit</button>
</form>

<div id='response-area2'>
</div>


<script>
    
document.getElementById('post-name-password').addEventListener('submit',(e)=> {
    e.preventDefault();
    
    const name = e.srcElement[0].value;
    const password = e.srcElement[1].value;
    const params = `post-name-password=&name=${name}&password=${password}`;
    
    const xhr = new XMLHttpRequest();
    xhr.open('POST', 'http://127.0.0.1/Javascript%20AJAX%20Testing/testerphp.php');
    xhr.setRequestHeader('Content-type', 'application/x-www-form-urlencoded');
    xhr.onload = ()=> {
        document.getElementById('response-area2').innerHTML = xhr.response;
    }
    xhr.send(params);
});

</script>


<br>

![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_green.png?raw=true)

## 3.3 - Fetch API

[Fetch API Documentation](https://developer.mozilla.org/en-US/docs/Web/API/Fetch_API)

<br>


* Fetch API is the newer interface compared to `XMLHttpRequest` for fetching resources, including across the network. 


* The main difference is that instead of using callback functions, it utilizes the newer ES6 feature of `Promise`. The `fetch()` function itself returns a `Promise` which we will use `.then()` and `.catch()` to process data


* The first parameter to `fetch()` is always the URL of the resource to be fetched. The second optional parameter will be a JS Object (called `init`) which specifies the settings of the AJAX request, including methods, headers etc. (See [This](https://developer.mozilla.org/en-US/docs/Web/API/WindowOrWorkerGlobalScope/fetch#Parameters) for full list of options)


* The data within the body of response of the request may need to be extracted. These are the methods for extraction, which also returns a `Promise`:
    * `arrayBuffer()` - Extracts into an arrayBuffer
    * `blob()` - Extracts into BLOB
    * `json()` - Extracts and parse into JSON
    * `text()` - Extracts into plain text
    * `formData()` - Extracts into FormData object

<br>

In [ ]:

%%js

//======================================
// Example for Text File (CSV) fetching
//======================================

async function fetchText() {
    const res = await fetch('https://raw.githubusercontent.com/AdmiJW/Miscellaneous-Items/master/fcc-forum-pageviews.csv');
    const text = await res.text();
    element.append( text.substr(0, 300).replaceAll('\r\n', '<br>') );
}

fetchText();


In [ ]:

%%js

//=================================================
// Example for BLOB fetching (Binary Large Objects)
//=================================================

async function fetchBlob() {
    const res = await fetch('https://freesound.org/data/previews/466/466998_285997-lq.mp3');
    const blob = await res.blob();
    console.log(blob);
    
    const audioHTML = document.createElement('audio');
    audioHTML.controls = true;
    element.append(audioHTML);
    
    const fr = new FileReader();
    fr.onload = ()=> audioHTML.src = fr.result;
    fr.readAsDataURL(blob);
}

fetchBlob();


In [ ]:
%%js

//=================================================
// Example for JSON 
//=================================================

async function fetchJSON() {
    const res = await fetch('https://jsonplaceholder.typicode.com/posts/1/comments');
    const json = await res.json();
    
    for (let i in json) {
        element.append(`<div><h3>${json[i].name}</h3><strong>${json[i].email}</strong><p>${json[i].body}</p></div>`)
    }
    
}

fetchJSON();


![Bar](https://github.com/AdmiJW/Items/blob/master/SeperatingBars/Horizontalbar_green.png?raw=true)

## 3.4 - Fetch with PHP


<br>

* Similar to `XMLHttpRequest`, `fetch` can also be used to retrieve data from web servers without having to refresh the page


* The small difference is that it is easier to set the header data in `fetch` compared to `XMLHttpRequest`


<br>



In [2]:
%%html

<!---------------------------------------------------
Example for PHP
------------------------------------------------------>

<form method='GET' id='echo_name2'>
    <input type='text' name='name' placeholder='Enter name here...'/>
    <button type='submit'>Submit</button>
</form>

<div id='response_area3'>
</div>



<script>
    
document.getElementById('echo_name2').addEventListener('submit', async (e) => {
    e.preventDefault();
    
    const name = e.srcElement[0].value;
    
    const res = await fetch('http://127.0.0.1/Javascript%20AJAX%20Testing/testerphp.php?echo-name=&name=' + name);  
    const text = await res.text();
    
    document.getElementById('response_area3').innerHTML = text;
});

</script>


In [12]:

%%html

<!---------------------------------------------------
Example for PHP
Sending POST request
------------------------------------------------------>

<form method='POST' id='post-name-password2'>
    <input type='text' name='name' placeholder='Enter username' required/>
    <input type='password' name='password' placeholder='Enter password' required/>
    <button type='submit' name='post-name-password' >Submit</button>
</form>

<div id='response-area4'>
</div>


<script>
    
document.getElementById("post-name-password2").addEventListener('submit', async (e)=> {
    e.preventDefault();

    const formData = new FormData(document.getElementById("post-name-password2") );
    formData.append('post-name-password', '');

    const res = await fetch('http://127.0.0.1/Javascript%20AJAX%20Testing/testerphp.php', {
        method: "POST",
        body: formData
    });
    const text = await res.text();
    
    document.getElementById('response-area4').innerHTML = text;
});

</script>
